# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 1:29PM,238088,10,CTF0011255,"Citieffe, Inc.",Released
1,May 25 2022 1:29PM,238088,10,CTF0011256,"Citieffe, Inc.",Released
2,May 25 2022 1:24PM,238086,19,ADV80002878,"AdvaGen Pharma, Ltd",Released
3,May 25 2022 1:24PM,238084,19,DEX0006865,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,May 25 2022 1:24PM,238084,19,DEX0006866,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,May 25 2022 1:14PM,238083,10,PRF-29955,Bio-PRF,Released
6,May 25 2022 1:14PM,238083,10,PRF-29958,Bio-PRF,Released
7,May 25 2022 1:14PM,238083,10,PRF-29962,Bio-PRF,Released
8,May 25 2022 1:14PM,238083,10,PRF-29965,Bio-PRF,Released
9,May 25 2022 1:14PM,238083,10,PRF-29968,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,238081,Released,25
30,238083,Released,11
31,238084,Released,2
32,238086,Released,1
33,238088,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238081,NaN,NaN,25.0
238083,NaN,NaN,11.0
238084,NaN,NaN,2.0
238086,NaN,NaN,1.0
238088,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0.0,16.0,19.0
238028,0.0,19.0,0.0
238033,0.0,0.0,2.0
238035,8.0,11.0,5.0
238037,4.0,15.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238021,0,16,19
238028,0,19,0
238033,0,0,2
238035,8,11,5
238037,4,15,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,0,16,19
1,238028,0,19,0
2,238033,0,0,2
3,238035,8,11,5
4,238037,4,15,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238021,,16,19
1,238028,,19,
2,238033,,,2
3,238035,8,11,5
4,238037,4,15,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 1:29PM,238088,10,"Citieffe, Inc."
2,May 25 2022 1:24PM,238086,19,"AdvaGen Pharma, Ltd"
3,May 25 2022 1:24PM,238084,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
5,May 25 2022 1:14PM,238083,10,Bio-PRF
16,May 25 2022 1:11PM,238081,10,"E5 Pharma, LLC"
41,May 25 2022 1:01PM,238080,22,"NBTY Global, Inc."
42,May 25 2022 12:53PM,238079,16,Seqirus
43,May 25 2022 12:39PM,238078,20,"ACI Healthcare USA, Inc."
50,May 25 2022 12:37PM,238075,10,ISDIN Corporation
85,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 1:29PM,238088,10,"Citieffe, Inc.",,,2
1,May 25 2022 1:24PM,238086,19,"AdvaGen Pharma, Ltd",,,1
2,May 25 2022 1:24PM,238084,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,2
3,May 25 2022 1:14PM,238083,10,Bio-PRF,,,11
4,May 25 2022 1:11PM,238081,10,"E5 Pharma, LLC",,,25
5,May 25 2022 1:01PM,238080,22,"NBTY Global, Inc.",,,1
6,May 25 2022 12:53PM,238079,16,Seqirus,,,1
7,May 25 2022 12:39PM,238078,20,"ACI Healthcare USA, Inc.",,,7
8,May 25 2022 12:37PM,238075,10,ISDIN Corporation,,15,20
9,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 1:29PM,238088,10,"Citieffe, Inc.",2,,
1,May 25 2022 1:24PM,238086,19,"AdvaGen Pharma, Ltd",1,,
2,May 25 2022 1:24PM,238084,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
3,May 25 2022 1:14PM,238083,10,Bio-PRF,11,,
4,May 25 2022 1:11PM,238081,10,"E5 Pharma, LLC",25,,
5,May 25 2022 1:01PM,238080,22,"NBTY Global, Inc.",1,,
6,May 25 2022 12:53PM,238079,16,Seqirus,1,,
7,May 25 2022 12:39PM,238078,20,"ACI Healthcare USA, Inc.",7,,
8,May 25 2022 12:37PM,238075,10,ISDIN Corporation,20,15,
9,May 25 2022 12:07PM,238068,19,"Granules Pharmaceuticals, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 1:29PM,238088,10,"Citieffe, Inc.",2,,
1,May 25 2022 1:24PM,238086,19,"AdvaGen Pharma, Ltd",1,,
2,May 25 2022 1:24PM,238084,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
3,May 25 2022 1:14PM,238083,10,Bio-PRF,11,,
4,May 25 2022 1:11PM,238081,10,"E5 Pharma, LLC",25,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 1:29PM,238088,10,"Citieffe, Inc.",2.0,NaN,NaN
1,May 25 2022 1:24PM,238086,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,May 25 2022 1:24PM,238084,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,NaN,NaN
3,May 25 2022 1:14PM,238083,10,Bio-PRF,11.0,NaN,NaN
4,May 25 2022 1:11PM,238081,10,"E5 Pharma, LLC",25.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 1:29PM,238088,10,"Citieffe, Inc.",2.0,0.0,0.0
1,May 25 2022 1:24PM,238086,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,May 25 2022 1:24PM,238084,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,0.0,0.0
3,May 25 2022 1:14PM,238083,10,Bio-PRF,11.0,0.0,0.0
4,May 25 2022 1:11PM,238081,10,"E5 Pharma, LLC",25.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2856788,94.0,39.0,6.0
15,476082,6.0,16.0,8.0
16,238079,1.0,0.0,0.0
18,476061,2.0,19.0,0.0
19,952280,3.0,2.0,0.0
20,476099,26.0,16.0,0.0
22,476130,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2856788,94.0,39.0,6.0
1,15,476082,6.0,16.0,8.0
2,16,238079,1.0,0.0,0.0
3,18,476061,2.0,19.0,0.0
4,19,952280,3.0,2.0,0.0
5,20,476099,26.0,16.0,0.0
6,22,476130,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,94.0,39.0,6.0
1,15,6.0,16.0,8.0
2,16,1.0,0.0,0.0
3,18,2.0,19.0,0.0
4,19,3.0,2.0,0.0
5,20,26.0,16.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,94.0
1,15,Released,6.0
2,16,Released,1.0
3,18,Released,2.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,22
Status,,,,,,,
Completed,6.0,8.0,0.0,0.0,0.0,0.0,0.0
Executing,39.0,16.0,0.0,19.0,2.0,16.0,0.0
Released,94.0,6.0,1.0,2.0,3.0,26.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,22
0,Completed,6.0,8.0,0.0,0.0,0.0,0.0,0.0
1,Executing,39.0,16.0,0.0,19.0,2.0,16.0,0.0
2,Released,94.0,6.0,1.0,2.0,3.0,26.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,22
0,Completed,6.0,8.0,0.0,0.0,0.0,0.0,0.0
1,Executing,39.0,16.0,0.0,19.0,2.0,16.0,0.0
2,Released,94.0,6.0,1.0,2.0,3.0,26.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()